In [ ]:
import datasets
from src import data_loading

dataset = datasets.load_dataset("lmarena-ai/arena-human-preference-140k")
data = data_loading.load_training_data(dataset["train"].to_pandas())
print(f"Successfully loaded {len(data.entries)} entries")

In [ ]:
from src.models.simple_scoring_model import SimpleScoringModel
from src.models.optimizers.adamw_spec import AdamWSpec
from src.utils.data_split import ValidationSplit


model = SimpleScoringModel(
    optimizer_spec=AdamWSpec(
        learning_rate=2,
        lr_decay_gamma=0.99,
        weight_decay=0.0,
        betas=(0, 0.999),
    ),
    tie_both_bad_epsilon=1e-3,
    non_ranking_loss_coeff=1e-3,
    balance_model_samples=True,
    print_every=5,
)

model.train(data, validation_split=ValidationSplit(val_fraction=0.2, seed=42), epochs=500, batch_size=2048)
model.save("simple-scoring-model-laptop-run")

print()
model.last_timer.inspect(1)

In [ ]:
import matplotlib.pyplot as plt

from src.plotting_utils import plot_loss, plot_accuracy

history = model.get_history()
_, axes = plt.subplots(3, 2, figsize=(15, 15))

plot_loss(axes[0, 0], history.total_loss, "Training loss")
plot_loss(axes[0, 1], history.val_loss, "Validation loss")
plot_accuracy(axes[1, 0], history.train_accuracy, "Training accuracy")
plot_accuracy(axes[1, 1], history.val_accuracy, "Validation accuracy")

axes[2, 0].plot(history.additional_metrics["ranking_accuracy"], label="Ranking accuracy")
axes[2, 0].plot(history.additional_metrics["tie_accuracy"], label="Tie accuracy")
axes[2, 0].plot(history.additional_metrics["both_bad_accuracy"], label="Both bad accuracy")
axes[2, 0].legend()

axes[2, 1].plot(history.additional_metrics["ranking_loss"], label="Ranking loss")
axes[2, 1].plot(history.additional_metrics["tie_loss"], label="Tie loss")
axes[2, 1].plot(history.additional_metrics["both_bad_loss"], label="Both bad loss")
axes[2, 1].legend()

plt.tight_layout()
plt.show()